In [9]:
import itertools
import numpy as np
import pandas as pd 
from numbers import Number
import sqlite3
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [10]:
gross = pd.read_csv('Data/bom.movie_gross.csv')

tmdb = pd.read_csv('Data/tmdb.movies.csv')

budget = pd.read_csv('Data/tn.movie_budgets.csv')
#budget = budget.set_index('id')

movie_info = pd.read_csv('Data/rt.movie_info.tsv', sep='\t')

reviews = pd.read_csv('Data/rt.reviews.tsv', sep='\t', encoding='windows-1252')

In [11]:
budget.id.value_counts()

id
1      58
63     58
61     58
60     58
59     58
       ..
86     57
85     57
84     57
83     57
100    57
Name: count, Length: 100, dtype: int64

In [12]:
gross['foreign_gross'].fillna(0, inplace = True)
gross['domestic_gross'].fillna(0, inplace = True)
gross['studio'].fillna('N/A', inplace = True)
gross.isna().sum()
gross_90s_on = gross[gross['year'] >= 1990]


gross_90s_on.info()
gross_90s_on = gross_90s_on.replace(',','', regex=True)
gross_90s_on.foreign_gross = gross_90s_on.foreign_gross.astype(np.float64)
money_made_gross_90s = gross_90s_on[(gross_90s_on['domestic_gross'] > 0) & (gross_90s_on['foreign_gross'] > 0)]
money_made_gross_90s

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3387 non-null   object 
 2   domestic_gross  3387 non-null   float64
 3   foreign_gross   3387 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000.0,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000.0,2010
3,Inception,WB,292600000.0,535700000.0,2010
4,Shrek Forever After,P/DW,238700000.0,513900000.0,2010
...,...,...,...,...,...
3275,I Still See You,LGF,1400.0,1500000.0,2018
3286,The Catcher Was a Spy,IFC,725000.0,229000.0,2018
3309,Time Freak,Grindstone,10000.0,256000.0,2018
3342,Reign of Judges: Title of Liberty - Concept Short,Darin Southa,93200.0,5200.0,2018


In [13]:
budget_90s_on = budget[budget.release_date >= '1990-01-01']
budget_90s_on[budget_90s_on.columns[3:]] = budget_90s_on[budget_90s_on.columns[3:]].replace('[\$,]', '', regex=True).astype(np.int64)

money_making_budget_90s_on = budget_90s_on[budget_90s_on['worldwide_gross'] > 0]
money_making_budget_90s_on


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747
...,...,...,...,...,...,...
5775,76,"May 26, 2006",Cavite,7000,70071,71644
5776,77,"Dec 31, 2004",The Mongol King,7000,900,900
5778,79,"Apr 2, 1999",Following,6000,48482,240495
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,1338


In [14]:
money_made_gross_90s

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000.0,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000.0,2010
3,Inception,WB,292600000.0,535700000.0,2010
4,Shrek Forever After,P/DW,238700000.0,513900000.0,2010
...,...,...,...,...,...
3275,I Still See You,LGF,1400.0,1500000.0,2018
3286,The Catcher Was a Spy,IFC,725000.0,229000.0,2018
3309,Time Freak,Grindstone,10000.0,256000.0,2018
3342,Reign of Judges: Title of Liberty - Concept Short,Darin Southa,93200.0,5200.0,2018


In [15]:
budget_gross_cleaned = pd.merge(money_made_gross_90s, money_making_budget_90s_on, how = 'outer', left_on = 'title', right_on = 'movie')

In [16]:
budget_gross_cleaned

,title,studio,domestic_gross_x,foreign_gross,year,id,release_date,movie,production_budget,domestic_gross_y,worldwide_gross
0,Toy Story 3,BV,415000000.0,652000000.0,2010.0,47.0,"Jun 18, 2010",Toy Story 3,200000000.0,415004880.0,1.068880e+09
1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000.0,2010.0,NaN,NaN,NaN,NaN,NaN,NaN
3,Inception,WB,292600000.0,535700000.0,2010.0,38.0,"Jul 16, 2010",Inception,160000000.0,292576195.0,8.355246e+08
4,Shrek Forever After,P/DW,238700000.0,513900000.0,2010.0,27.0,"May 21, 2010",Shrek Forever After,165000000.0,238736787.0,7.562447e+08
...,...,...,...,...,...,...,...,...,...,...,...
6355,NaN,NaN,NaN,NaN,NaN,76.0,"May 26, 2006",Cavite,7000.0,70071.0,7.164400e+04
6356,NaN,NaN,NaN,NaN,NaN,77.0,"Dec 31, 2004",The Mongol King,7000.0,900.0,9.000000e+02
6357,NaN,NaN,NaN,NaN,NaN,79.0,"Apr 2, 1999",Following,6000.0,48482.0,2.404950e+05
6358,NaN,NaN,NaN,NaN,NaN,80.0,"Jul 13, 2005",Return to the Land of Wonders,5000.0,1338.0,1.338000e+03


In [17]:
cleaned = budget_gross_cleaned.title.fillna(budget_gross_cleaned.movie)
budget_gross_cleaned.title = cleaned
budget_gross_cleaned.isna().sum()
budget_gross_cleaned = budget_gross_cleaned.drop('movie', axis=1)
cleaned

0                                       Toy Story 3
1                        Alice in Wonderland (2010)
2       Harry Potter and the Deathly Hallows Part 1
3                                         Inception
4                               Shrek Forever After
                           ...                     
6355                                         Cavite
6356                                The Mongol King
6357                                      Following
6358                  Return to the Land of Wonders
6359                              My Date With Drew
Name: title, Length: 6360, dtype: object

In [18]:
budget_gross_cleaned.isna().sum()

title                   0
studio               4345
domestic_gross_x     4345
foreign_gross        4345
year                 4345
id                    945
release_date          945
production_budget     945
domestic_gross_y      945
worldwide_gross       945
dtype: int64

In [19]:
cleaned = budget_gross_cleaned.domestic_gross_y.fillna(budget_gross_cleaned.domestic_gross_x)
budget_gross_cleaned.domestic_gross_y = cleaned
budget_gross_cleaned.isna().sum()

title                   0
studio               4345
domestic_gross_x     4345
foreign_gross        4345
year                 4345
id                    945
release_date          945
production_budget     945
domestic_gross_y        0
worldwide_gross       945
dtype: int64

In [20]:
budget_gross_cleaned = budget_gross_cleaned.drop('domestic_gross_x', axis=1)


In [21]:
budget_gross_cleaned

,title,studio,foreign_gross,year,id,release_date,production_budget,domestic_gross_y,worldwide_gross
0,Toy Story 3,BV,652000000.0,2010.0,47.0,"Jun 18, 2010",200000000.0,415004880.0,1.068880e+09
1,Alice in Wonderland (2010),BV,691300000.0,2010.0,NaN,NaN,NaN,334200000.0,NaN
2,Harry Potter and the Deathly Hallows Part 1,WB,664300000.0,2010.0,NaN,NaN,NaN,296000000.0,NaN
3,Inception,WB,535700000.0,2010.0,38.0,"Jul 16, 2010",160000000.0,292576195.0,8.355246e+08
4,Shrek Forever After,P/DW,513900000.0,2010.0,27.0,"May 21, 2010",165000000.0,238736787.0,7.562447e+08
...,...,...,...,...,...,...,...,...,...
6355,Cavite,NaN,NaN,NaN,76.0,"May 26, 2006",7000.0,70071.0,7.164400e+04
6356,The Mongol King,NaN,NaN,NaN,77.0,"Dec 31, 2004",7000.0,900.0,9.000000e+02
6357,Following,NaN,NaN,NaN,79.0,"Apr 2, 1999",6000.0,48482.0,2.404950e+05
6358,Return to the Land of Wonders,NaN,NaN,NaN,80.0,"Jul 13, 2005",5000.0,1338.0,1.338000e+03


In [22]:
budget_gross_cleaned = budget_gross_cleaned[budget_gross_cleaned['production_budget'].notna()]

In [23]:
budget_gross_cleaned.isna().sum()

title                   0
studio               4345
foreign_gross        4345
year                 4345
id                      0
release_date            0
production_budget       0
domestic_gross_y        0
worldwide_gross         0
dtype: int64

In [24]:
cleaned = budget_gross_cleaned.foreign_gross.fillna(budget_gross_cleaned.worldwide_gross - budget_gross_cleaned.domestic_gross_y )
budget_gross_cleaned.foreign_gross = cleaned

In [25]:
budget_gross_cleaned.isna().sum()

title                   0
studio               4345
foreign_gross           0
year                 4345
id                      0
release_date            0
production_budget       0
domestic_gross_y        0
worldwide_gross         0
dtype: int64

In [26]:
budget_gross_cleaned

,title,studio,foreign_gross,year,id,release_date,production_budget,domestic_gross_y,worldwide_gross
0,Toy Story 3,BV,652000000.0,2010.0,47.0,"Jun 18, 2010",200000000.0,415004880.0,1.068880e+09
3,Inception,WB,535700000.0,2010.0,38.0,"Jul 16, 2010",160000000.0,292576195.0,8.355246e+08
4,Shrek Forever After,P/DW,513900000.0,2010.0,27.0,"May 21, 2010",165000000.0,238736787.0,7.562447e+08
5,The Twilight Saga: Eclipse,Sum.,398000000.0,2010.0,53.0,"Jun 30, 2010",68000000.0,300531751.0,7.061028e+08
6,Iron Man 2,Par.,311500000.0,2010.0,15.0,"May 7, 2010",170000000.0,312433331.0,6.211564e+08
...,...,...,...,...,...,...,...,...,...
6355,Cavite,NaN,1573.0,NaN,76.0,"May 26, 2006",7000.0,70071.0,7.164400e+04
6356,The Mongol King,NaN,0.0,NaN,77.0,"Dec 31, 2004",7000.0,900.0,9.000000e+02
6357,Following,NaN,192013.0,NaN,79.0,"Apr 2, 1999",6000.0,48482.0,2.404950e+05
6358,Return to the Land of Wonders,NaN,0.0,NaN,80.0,"Jul 13, 2005",5000.0,1338.0,1.338000e+03


In [27]:
budget_gross_cleaned.isna().sum()

title                   0
studio               4345
foreign_gross           0
year                 4345
id                      0
release_date            0
production_budget       0
domestic_gross_y        0
worldwide_gross         0
dtype: int64

In [28]:
titles = list(budget_gross_cleaned.title)

In [29]:
tmdb = tmdb[tmdb['original_title'].isin(titles)]


In [30]:
budget_gross_cleaned['domestic_success'] = (budget_gross_cleaned['domestic_gross_y']/budget_gross_cleaned['production_budget'])

budget_gross_cleaned['foreign_success'] = (budget_gross_cleaned['foreign_gross']/budget_gross_cleaned['production_budget'])

budget_gross_cleaned['WW_success'] = (budget_gross_cleaned['worldwide_gross']/budget_gross_cleaned['production_budget'])
                                        

In [31]:
budget_gross_cleaned.sort_values('id')

,title,studio,foreign_gross,year,id,release_date,production_budget,domestic_gross_y,worldwide_gross,domestic_success,foreign_success,WW_success
2405,Timeline,NaN,7222445.0,NaN,1.0,"Nov 26, 2003",80000000.0,19480739.0,26703184.0,0.243509,0.090281,0.333790
1092,Dumb and Dumber To,Uni.,83600000.0,2014.0,1.0,"Nov 14, 2014",40000000.0,86208010.0,156553592.0,2.155200,2.090000,3.913840
3234,Jersey Girl,NaN,11800000.0,NaN,1.0,"Mar 26, 2004",35000000.0,25266129.0,37066129.0,0.721889,0.337143,1.059032
1074,Mr. Peabody & Sherman,Fox,164200000.0,2014.0,1.0,"Mar 7, 2014",145000000.0,111506430.0,269806430.0,0.769010,1.132414,1.860734
5893,Stolen Summer,NaN,0.0,NaN,1.0,"Mar 22, 2002",1500000.0,119841.0,119841.0,0.079894,0.000000,0.079894
...,...,...,...,...,...,...,...,...,...,...,...,...
645,The Impossible,LG/S,161300000.0,2012.0,100.0,"Dec 21, 2012",40000000.0,19019882.0,169590606.0,0.475497,4.032500,4.239765
4884,The Ultimate Gift,NaN,0.0,NaN,100.0,"Mar 9, 2007",9000000.0,3438735.0,3438735.0,0.382082,0.000000,0.382082
5598,Teen Wolf Too,NaN,0.0,NaN,100.0,"Nov 20, 1987",3000000.0,7888000.0,7888000.0,2.629333,0.000000,2.629333
4800,Parkland,NaN,974914.0,NaN,100.0,"Oct 4, 2013",10000000.0,641439.0,1616353.0,0.064144,0.097491,0.161635


In [32]:
domestic_cleaned = budget_gross_cleaned[budget_gross_cleaned['domestic_success'] > 1]

In [33]:
domestic_cleaned

,title,studio,foreign_gross,year,id,release_date,production_budget,domestic_gross_y,worldwide_gross,domestic_success,foreign_success,WW_success
0,Toy Story 3,BV,652000000.0,2010.0,47.0,"Jun 18, 2010",200000000.0,415004880.0,1.068880e+09,2.075024,3.260000,5.344398
3,Inception,WB,535700000.0,2010.0,38.0,"Jul 16, 2010",160000000.0,292576195.0,8.355246e+08,1.828601,3.348125,5.222029
4,Shrek Forever After,P/DW,513900000.0,2010.0,27.0,"May 21, 2010",165000000.0,238736787.0,7.562447e+08,1.446890,3.114545,4.583301
5,The Twilight Saga: Eclipse,Sum.,398000000.0,2010.0,53.0,"Jun 30, 2010",68000000.0,300531751.0,7.061028e+08,4.419585,5.852941,10.383865
6,Iron Man 2,Par.,311500000.0,2010.0,15.0,"May 7, 2010",170000000.0,312433331.0,6.211564e+08,1.837843,1.832353,3.653861
...,...,...,...,...,...,...,...,...,...,...,...,...
6353,El Mariachi,NaN,1008.0,NaN,74.0,"Feb 26, 1993",7000.0,2040920.0,2.041928e+06,291.560000,0.144000,291.704000
6354,Primer,NaN,417166.0,NaN,75.0,"Oct 8, 2004",7000.0,424760.0,8.419260e+05,60.680000,59.595143,120.275143
6355,Cavite,NaN,1573.0,NaN,76.0,"May 26, 2006",7000.0,70071.0,7.164400e+04,10.010143,0.224714,10.234857
6357,Following,NaN,192013.0,NaN,79.0,"Apr 2, 1999",6000.0,48482.0,2.404950e+05,8.080333,32.002167,40.082500


In [34]:
foreign_cleaned = budget_gross_cleaned[budget_gross_cleaned['foreign_success'] > 1]
foreign_cleaned

,title,studio,foreign_gross,year,id,release_date,production_budget,domestic_gross_y,worldwide_gross,domestic_success,foreign_success,WW_success
0,Toy Story 3,BV,652000000.0,2010.0,47.0,"Jun 18, 2010",200000000.0,415004880.0,1.068880e+09,2.075024,3.260000,5.344398
3,Inception,WB,535700000.0,2010.0,38.0,"Jul 16, 2010",160000000.0,292576195.0,8.355246e+08,1.828601,3.348125,5.222029
4,Shrek Forever After,P/DW,513900000.0,2010.0,27.0,"May 21, 2010",165000000.0,238736787.0,7.562447e+08,1.446890,3.114545,4.583301
5,The Twilight Saga: Eclipse,Sum.,398000000.0,2010.0,53.0,"Jun 30, 2010",68000000.0,300531751.0,7.061028e+08,4.419585,5.852941,10.383865
6,Iron Man 2,Par.,311500000.0,2010.0,15.0,"May 7, 2010",170000000.0,312433331.0,6.211564e+08,1.837843,1.832353,3.653861
...,...,...,...,...,...,...,...,...,...,...,...,...
6324,George Washington,NaN,100906.0,NaN,28.0,"Oct 27, 2000",42000.0,241816.0,3.427220e+05,5.757524,2.402524,8.160048
6334,Clerks,NaN,820812.0,NaN,43.0,"Oct 19, 1994",27000.0,3073428.0,3.894240e+06,113.830667,30.400444,144.231111
6340,Exeter,NaN,489792.0,NaN,49.0,"Sep 1, 2015",25000.0,0.0,4.897920e+05,0.000000,19.591680,19.591680
6354,Primer,NaN,417166.0,NaN,75.0,"Oct 8, 2004",7000.0,424760.0,8.419260e+05,60.680000,59.595143,120.275143


In [35]:
WW_cleaned =  budget_gross_cleaned[budget_gross_cleaned['WW_success'] > 1]
WW_cleaned

,title,studio,foreign_gross,year,id,release_date,production_budget,domestic_gross_y,worldwide_gross,domestic_success,foreign_success,WW_success
0,Toy Story 3,BV,652000000.0,2010.0,47.0,"Jun 18, 2010",200000000.0,415004880.0,1.068880e+09,2.075024,3.260000,5.344398
3,Inception,WB,535700000.0,2010.0,38.0,"Jul 16, 2010",160000000.0,292576195.0,8.355246e+08,1.828601,3.348125,5.222029
4,Shrek Forever After,P/DW,513900000.0,2010.0,27.0,"May 21, 2010",165000000.0,238736787.0,7.562447e+08,1.446890,3.114545,4.583301
5,The Twilight Saga: Eclipse,Sum.,398000000.0,2010.0,53.0,"Jun 30, 2010",68000000.0,300531751.0,7.061028e+08,4.419585,5.852941,10.383865
6,Iron Man 2,Par.,311500000.0,2010.0,15.0,"May 7, 2010",170000000.0,312433331.0,6.211564e+08,1.837843,1.832353,3.653861
...,...,...,...,...,...,...,...,...,...,...,...,...
6353,El Mariachi,NaN,1008.0,NaN,74.0,"Feb 26, 1993",7000.0,2040920.0,2.041928e+06,291.560000,0.144000,291.704000
6354,Primer,NaN,417166.0,NaN,75.0,"Oct 8, 2004",7000.0,424760.0,8.419260e+05,60.680000,59.595143,120.275143
6355,Cavite,NaN,1573.0,NaN,76.0,"May 26, 2006",7000.0,70071.0,7.164400e+04,10.010143,0.224714,10.234857
6357,Following,NaN,192013.0,NaN,79.0,"Apr 2, 1999",6000.0,48482.0,2.404950e+05,8.080333,32.002167,40.082500


In [36]:
low_budget = budget_gross_cleaned[budget_gross_cleaned['production_budget'] < 5000000]
mid_budget = budget_gross_cleaned[(budget_gross_cleaned['production_budget'] > 5000000) & (budget_gross_cleaned['production_budget'] < 50000000)]
high_budget = budget_gross_cleaned[budget_gross_cleaned['production_budget'] > 50000000]

In [37]:
WW_cleaned

,title,studio,foreign_gross,year,id,release_date,production_budget,domestic_gross_y,worldwide_gross,domestic_success,foreign_success,WW_success
0,Toy Story 3,BV,652000000.0,2010.0,47.0,"Jun 18, 2010",200000000.0,415004880.0,1.068880e+09,2.075024,3.260000,5.344398
3,Inception,WB,535700000.0,2010.0,38.0,"Jul 16, 2010",160000000.0,292576195.0,8.355246e+08,1.828601,3.348125,5.222029
4,Shrek Forever After,P/DW,513900000.0,2010.0,27.0,"May 21, 2010",165000000.0,238736787.0,7.562447e+08,1.446890,3.114545,4.583301
5,The Twilight Saga: Eclipse,Sum.,398000000.0,2010.0,53.0,"Jun 30, 2010",68000000.0,300531751.0,7.061028e+08,4.419585,5.852941,10.383865
6,Iron Man 2,Par.,311500000.0,2010.0,15.0,"May 7, 2010",170000000.0,312433331.0,6.211564e+08,1.837843,1.832353,3.653861
...,...,...,...,...,...,...,...,...,...,...,...,...
6353,El Mariachi,NaN,1008.0,NaN,74.0,"Feb 26, 1993",7000.0,2040920.0,2.041928e+06,291.560000,0.144000,291.704000
6354,Primer,NaN,417166.0,NaN,75.0,"Oct 8, 2004",7000.0,424760.0,8.419260e+05,60.680000,59.595143,120.275143
6355,Cavite,NaN,1573.0,NaN,76.0,"May 26, 2006",7000.0,70071.0,7.164400e+04,10.010143,0.224714,10.234857
6357,Following,NaN,192013.0,NaN,79.0,"Apr 2, 1999",6000.0,48482.0,2.404950e+05,8.080333,32.002167,40.082500


In [38]:
low_budget

,title,studio,foreign_gross,year,id,release_date,production_budget,domestic_gross_y,worldwide_gross,domestic_success,foreign_success,WW_success
41,Paranormal Activity 2,Par.,92800000.0,2010.0,65.0,"Oct 20, 2010",3000000.0,84752907.0,177512032.0,28.250969,30.933333,59.170677
89,The Last Exorcism,LGF,26700000.0,2010.0,15.0,"Aug 27, 2010",1800000.0,41034350.0,70165900.0,22.796861,14.833333,38.981056
127,The Kids Are All Right,Focus,13900000.0,2010.0,87.0,"Jul 9, 2010",4000000.0,20811365.0,36275469.0,5.202841,3.475000,9.068867
149,Buried,LGF,18100000.0,2010.0,2.0,"Sep 24, 2010",1987650.0,1044143.0,21270290.0,0.525315,9.106231,10.701225
153,Soul Kitchen,IFC,17600000.0,2010.0,3.0,"Aug 20, 2010",4900000.0,276901.0,23347461.0,0.056510,3.591837,4.764788
...,...,...,...,...,...,...,...,...,...,...,...,...
6355,Cavite,NaN,1573.0,NaN,76.0,"May 26, 2006",7000.0,70071.0,71644.0,10.010143,0.224714,10.234857
6356,The Mongol King,NaN,0.0,NaN,77.0,"Dec 31, 2004",7000.0,900.0,900.0,0.128571,0.000000,0.128571
6357,Following,NaN,192013.0,NaN,79.0,"Apr 2, 1999",6000.0,48482.0,240495.0,8.080333,32.002167,40.082500
6358,Return to the Land of Wonders,NaN,0.0,NaN,80.0,"Jul 13, 2005",5000.0,1338.0,1338.0,0.267600,0.000000,0.267600


In [39]:
type(low_budget['genres'].iloc[1])

KeyError: 'genres'